In [ ]:
import sys
from pathlib import Path

def add_project_root(levels_up: int = 1):
    """Adds the project root to sys.path for Jupyter or scripts."""
    root = Path().resolve()
    for _ in range(levels_up):
        root = root.parent
    if str(root) not in sys.path:
        sys.path.append(str(root))
add_project_root() 

In [ ]:
import pandas as pd
from transformers import pipeline
import evaluate

# from modules.pipeline import complete_pipeline, claim_judge_pipeline, data_preparation, generate_summary_database
from modules.summarizer import Summarizer
from modules.SummaryManager import EvalSummaryManager
from modules.summary_enhancer import enhancer
from modules.data_handler import load_discharges, load_patient_data, load_clinical_context
# from modules.context_handler import context_builder_text, context_builder_json, build_discharge_windows, get_patient_window_data

In [ ]:
DECOMPOSITION_PROMPT_TEMPLATE = '''

You are a medical information analyst. Your task is to parse the entire medical summary below and decompose it into a complete list of fine-grained, verifiable claims. These claims will be used to fact-check the summary against a full patient record.

Each claim must be a single, self-contained medical fact, finding, instruction, or patient-reported statement.

**Extraction Rules:**
1.  **Split Lists:** Decompose lists of conditions, symptoms, findings, or plans into separate claims (e.g., "history of A, B, and C" becomes three separate claims).
2.  **Keep Instructions Intact:** A single medication (e.g., "Furosemide 40mg daily") is one complete claim. Do not split the drug from its dosage.
3.  **Be Self-Contained:** Resolve all pronouns (e.g., "She," "He") to "The patient."
4.  **Extract All Details:** Capture demographics, past medical history (PMH), symptoms, lab results, imaging findings, diagnoses, medications, and follow-up plans.
5.  **Output Format:** Provide the output as a simple list of claims one in each line.

---
### Example 1

**Full Summary:**
The patient, a female with a history of HCV cirrhosis with ascites, HIV on ART, IV drug use, COPD, bipolar disorder, and PTSD, presented with worsening abdominal distension and pain. She had discontinued her Lasix and Spironolactone prior to admission due to feeling that they weren't effective and wanting to avoid "chemicals."During hospitalization, the patient's ascites was managed with diuretics (Furosemide 40mg and Spironolactone 50mg daily). Paracentesis was attempted in the ED but was unsuccessful. Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili), and a low platelet count. A CXR showed no acute cardiopulmonary process. Ultrasound revealed liver nodularity, signs of portal hypertension (ascites and splenomegaly), and cholelithiasis. The patient was scheduled for follow-up with her current PCP, a new PCP (Dr. ___), and the Liver Clinic for EGD and ___. She was discharged home in a clear and coherent mental state, alert, interactive, and ambulatory, with a diagnosis of ascites from portal hypertension. She was instructed to continue taking Furosemide 40mg and Spironolactone 50mg daily, adhere to a low-salt diet, and follow up with the Liver Clinic.


**Claims:**
The patient is female.

The patient has a history of HCV cirrhosis with ascites.

The patient has a history of HIV on ART.

The patient has a history of IV drug use.

The patient has a history of COPD.

The patient has a history of bipolar disorder.

The patient has a history of PTSD.

The patient presented with worsening abdominal distension.

The patient presented with worsening abdominal pain.

The patient had discontinued her Lasix prior to admission.

The patient had discontinued her Spironolactone prior to admission.

The patient's reason for discontinuing was feeling [the drugs] weren't effective.

The patient's reason for discontinuing was wanting to avoid "chemicals."

The patient's ascites was managed with Furosemide 40mg daily during hospitalization.

The patient's ascites was managed with Spironolactone 50mg daily during hospitalization.

Paracentesis was attempted in the ED.

The paracentesis attempt in the ED was unsuccessful.

Lab results showed elevated liver enzymes (ALT, AST, Alk Phos, Total Bili).

Lab results showed a low platelet count.

A CXR showed no acute cardiopulmonary process.

Ultrasound revealed liver nodularity.

Ultrasound revealed signs of portal hypertension (ascites and splenomegaly).

Ultrasound revealed cholelithiasis.

The patient was scheduled for follow-up with her current PCP.

The patient was scheduled for follow-up with a new PCP (Dr. ___).

The patient was scheduled for follow-up with the Liver Clinic for EGD.

The patient was scheduled for follow-up with the Liver Clinic for ___.

The patient was discharged home.

The patient's mental state at discharge was clear and coherent.

The patient's status at discharge was alert.

The patient's status at discharge was interactive.

The patient's status at discharge was ambulatory.

The patient's diagnosis was ascites from portal hypertension.

The patient was instructed to continue taking Furosemide 40mg daily.

The patient was instructed to continue taking Spironolactone 50mg daily.

The patient was instructed to adhere to a low-salt diet.

The patient was instructed to follow up with the Liver Clinic.

---
### New summary (YOUR TASK)

**Full Summary:**
{summary_text}
'''


In [ ]:
prompt_template = '''

    You are a medical analyst and fact-checker. Your job is to judge clinical claims about a specific patient by carefully reading only the text provided (discharge note and related context sections). You must not use any outside knowledge or invent facts beyond the provided text. Focus strictly on whether the CLAIM is supported, contradicted, or cannot be determined from the supplied text. Your output must be strictly evidence-grounded and follow the exact output format described below.

    INSTRUCTIONS (read carefully):
    1) You are given: (A) a discharge note and other context sections from the patient's chart, (B) a single CLAIM to judge.
    2) Use ONLY the provided text as evidence. Do NOT use external knowledge or make assumptions that are not supported by the text.
    3) Look for explicit statements and strong implications in the text that support or contradict the claim. If the text only *lists medications* in a discharge-medications section, treat that as a strong implication of continuation unless there is explicit contradictory language (e.g., "discontinued", "stopped", "hold", "do not resume") in the discharge or plan.
    4) For every judgment you make, quote the exact sentence(s) or contiguous text span(s) from the provided text that you used as evidence. Put each quoted piece on its own line (a quoted block per evidence piece).
    5) If more than one separate section supports/contradicts the claim, include each supporting/contradicting quote on separate lines in the order you used them.
    6) After the quoted evidence lines, output a single line with one token LABEL which must be exactly one of:
    - ENTAILMENT
    - CONTRADICTION
    - NEUTRAL
    7) Do not output anything else — no preamble, no extra commentary, and no hallucinated citations.

    ------------------------------------------------------------
    FEW-SHOT EXAMPLES
    ------------------------------------------------------------

    ### EXAMPLE 1
    CLAIM:
    "The patient was advised to continue her HIV medications at discharge."

    EVIDENCE:
    "Discharge Medications: ... 2. Emtricitabine-Tenofovir (Truvada) 1 TAB PO DAILY ... 6. Raltegravir 400 mg PO BID"
    "Dear Ms. ___, ... Please take these medications daily to keep excess fluid off ..."
    ENTAILMENT

    ---

    ### EXAMPLE 2
    CLAIM:
    "The patient was confused at the time of discharge."

    EVIDENCE:
    "Discharge Condition: Mental Status: Clear and coherent. Level of Consciousness: Alert and interactive."
    CONTRADICTION

    ---

    ### EXAMPLE 3
    CLAIM:
    "A successful paracentesis was performed during the hospitalization."

    EVIDENCE:
    "Diagnostic para attempted in the ED, unsuccessful."
    "Discharge Instructions: ... we did a paracentesis to remove 1.5L of fluid from your belly."
    ENTAILMENT

    ------------------------------------------------------------
    END OF EXAMPLES
    ------------------------------------------------------------


    --- 
    INPUT:
    [DISCHARGE NOTE AND CONTEXT SECTIONS — keep section headers]
    {premise}

    CLAIM:
    {claim}
'''

In [ ]:
discharge = '''

 
Name:  ___.          Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Tramadol
 
Attending: ___.
 
Chief Complaint:
Abdominal distention, back pain, fever; leukocytosis.
 
Major Surgical or Invasive Procedure:
Paracentesis x 3.

 
History of Present Illness:
This is a ___ woman with a history of ETOH abuse who 
presents with abdominal distention, back pain, fever, and 
elevated white count from Liver Clinic.  Ms. ___ was 
recently admitted to this hospital about 1 week ago for 
treatment of ascites and work-up of alcoholic hepatitis.  At 
that time she had a diagnostic and therapeutic paracentesis and 
was treated for a UTI.  She was discharged home and instructed 
to follow-up in Liver Clinic in 1 week.  On day of presentation 
to liver clinic, patient complained of worsening abdominal pain 
and low-grade fevers at home.  Her labwork was also significant 
for an elevated white count.  As such, Ms. ___ was 
admitted for work-up of fever and white count, and for treatment 
of recurrent ascites.
 
Past Medical History:
--Alcohol abuse
--Chronic back pain
 
Social History:
___
Family History:
Breast cancer in mother age ___, No IBD, liver failure.  Multiple 
relatives with alcoholism.
 
Physical Exam:
VS: 97.9, 103/73, 86, 18, 96% RA  
GEN: A/Ox3, pleasant, appropriate, well appearing  
HEENT: No temporal wasting, JVD not elevated, neck veins fill 
from above.  
CV: RRR, No MRG  
PULM: CTAB but decreased BS in R base.  
ABD: Distended and tight, diffusely tender to palpation, BS+, + 
passing flatulence.  
LIMBS: 2+ edema of the LEs to knee bilaterally ___ pulses 2+ 
bilaterally  
NEURO: No asterixis, very mild general tremor.
 
Pertinent Results:
Labs at Admission:

___ 09:47AM BLOOD WBC-26.2*# RBC-3.86* Hgb-13.0 Hct-43.3 
MCV-112* MCH-33.7* MCHC-30.0* RDW-12.7 Plt ___
___ 09:47AM BLOOD Neuts-88* Bands-1 Lymphs-2* Monos-7 Eos-1 
Baso-1 ___ Myelos-0
___ 09:20PM BLOOD ___
___ 09:47AM BLOOD UreaN-8 Creat-0.5 Na-133 K-5.1 Cl-92* 
HCO3-26 AnGap-20
___ 09:47AM BLOOD ALT-45* AST-165* LD(LDH)-345* 
AlkPhos-200* TotBili-2.0*
___ 09:47AM BLOOD Albumin-2.9* Calcium-8.1* Phos-4.0 Mg-2.2
___ 09:20PM BLOOD Ethanol-NEG Bnzodzp-NEG

Labs at Discharge:

___ 07:20AM BLOOD WBC-20.7* RBC-3.03* Hgb-10.3* Hct-32.0* 
MCV-106* MCH-33.9* MCHC-32.1 RDW-13.7 Plt ___
___ 07:20AM BLOOD ___ PTT-42.0* ___
___ 07:20AM BLOOD Glucose-96 UreaN-7 Creat-0.4 Na-125* 
K-4.4 Cl-90* HCO3-30 AnGap-9
___ 07:20AM BLOOD ALT-35 AST-131* LD(___)-265* AlkPhos-184* 
TotBili-1.9*
___ 07:20AM BLOOD Albumin-2.5* Calcium-7.2* Phos-2.6* 
Mg-2.0

Micro Data:

___ PERITONEAL FLUID GRAM STAIN- negative; FLUID 
CULTURE-PENDING; ANAEROBIC CULTURE- negative
___ STOOL CLOSTRIDIUM DIFFICILE TOXIN A & B 
TEST- negative
___ URINE URINE CULTURE- negative
___ SWAB R/O VANCOMYCIN RESISTANT ENTEROCOCCUS- 
negative
___ STOOL CLOSTRIDIUM DIFFICILE TOXIN A & B 
TEST- negative
___ FLUID RECEIVED IN BLOOD CULTURE BOTTLES 
Fluid Culture in Bottles- negative
___ PERITONEAL FLUID GRAM STAIN-FINAL; FLUID 
CULTURE-FINAL; ANAEROBIC CULTURE- negative
___ BLOOD CULTURE Blood Culture, Routine- 
negative 
___ BLOOD CULTURE Blood Culture, 
Routine-negative 
___ URINE URINE CULTURE-FINAL {GRAM POSITIVE 
BACTERIA} INPATIENT 
___ FLUID RECEIVED IN BLOOD CULTURE BOTTLES 
negative

Imaging Results:

CTA (___):
1. No evidence of pulmonary embolism. 
2. Stable atelectasis at the right lung base. 
3. Moderate right and small left pleural effusions, unchanged. 

CTAP (___):
1. Hepatomegaly and large ascites consistent with stated history 
of liver 
disease. No evidence of portal venous thrombosis suggesting that 
the findings on the prior ultrasound may have resulted from 
extremely slow / undetectable flow.  
2. Moderate right and small left pleural effusions, increased on 
the right 
with right basilar atelectasis. 
3. Replaced right hepatic artery arising from the SMA, otherwise 
conventional arterial and venous anatomy.
 
Brief Hospital Course:
This is a ___ woman with likely alcoholic hepatitis and 
recurrent ascites who is admitted with low-grade fevers, high 
white count, and abdominal pain.
 
# ASCITES/ALC HEPATITIS/LEUKOCYTOSIS: Patient with fatty liver 
and ascites in setting of extensive drinking history and AST/ALT 
elevation >2. Discriminant function on admission was ~30.  
Patient had a paracentesis on ___ and 4L was removed; 
peritoneal fluid was negative for SBP. Diuretics were initially 
held in the setting of hyponatremia. She was treated 
supportively with nutrition, brief antibiotics for urinary tract 
infection (3-days of ceftriaxone), and therapeutic paracenteses 
x3. Her symptoms, white cell count, and total bilirubin were 
improving at time of discharge. She will follow-up with Dr. 
___ in liver clinic and with her primary care provider, Dr. 
___, in two weeks. 

# HYPONATREMIA: Likely hypovolemic hyponatremia with some 
component of euvolemic hyponatremia from liver disease. Her 
spironlactone was held and can be restarted at the discretion of 
her outpatient liver team, if necessary. Sodium at time of 
discharge was 125. She has been advised to continue a low sodium 
diet and free water restriction to ___ liters daily.
 
# ALCOHOLISM: Patient has been trying to cut back recently, but 
reports daily heavy alcohol intake for the past ___ years; she has 
had withdrawal symptoms before but no seizures. Shakes and 
hallucinations.  Reports sobriety since prior admission. She 
will continue outpatient rehab.
  
# URINARY TRACT INFECTION: she was treated with a three-day 
course of empiric ceftriaxone for concern of UTI.

# BACK PAIN/ABDOMINAL PAIN: this was treated in house with 
lidocaine patches as needed and oxycodone as needed. She has 
been provided with a short course of Tramadol to take as needed 
until follow-up with her primary care provider. She understands 
that this is only a temporary medication and will be 
discontinued when her acute hepatitis resolves.

# Prophylaxis:  
-DVT ppx with SC heparin  
-Bowel regimen with lactulose, no PPI  
-Pain management with oxycodone and lidocaine patch  
# Communication: Patient  
# Code: presumed full
 
Medications on Admission:
Multivitamin, thiamine, folate, spironolactone 25mg daily, 
lidocaine patch prn, nicotine patch.
 
Discharge Medications:
1. Multivitamin     Tablet Sig: One (1) Tablet PO DAILY (Daily). 
 
2. Thiamine HCl 100 mg Tablet Sig: One (1) Tablet PO DAILY 
(Daily).  
3. Folic Acid 1 mg Tablet Sig: One (1) Tablet PO DAILY (Daily).  

4. Nicotine 14 mg/24 hr Patch 24 hr Sig: One (1) Patch 24 hr 
Transdermal DAILY (Daily).  
5. Lidocaine 5 %(700 mg/patch) Adhesive Patch, Medicated Sig: 
One (1) Adhesive Patch, Medicated Topical DAILY (Daily).  
6. Tramadol 50 mg Tablet Sig: One (1) Tablet PO every eight (8) 
hours as needed for pain.
Disp:*30 Tablet(s)* Refills:*0*

 
Discharge Disposition:
Home
 
Discharge Diagnosis:
Primary diagnosis: Alcoholic hepatitis.
 
Discharge Condition:
Mental Status: Clear and coherent.
Level of Consciousness: Alert and interactive.
Activity Status: Ambulatory - Independent.
 
Discharge Instructions:
You were admitted to the hospital for alcoholic hepatitis. This 
is a condition in which your liver becomes inflamed due to 
excessive alcohol intake. You were also noted to have an 
elevated white cell count which can sometimes indicate 
infection. You were treated with a brief course of antibiotics 
for a urinary tract infection. Otherwise your blood and 
peritoneal fluid cultures remain negative.

We made the following changes to your medications: 
We stopped your spironolactone because your blood sodium levels 
were too low.
We added Tramadol to take as needed for back pain. 
 
Followup Instructions:
___


'''

In [ ]:
summarizer.model.count_tokens(prompt_template)

In [ ]:

# Preload the NLI model once (outside function for efficiency)
nli = pipeline("text-classification", model="facebook/bart-large-mnli")


def decomp_judge_subpipeline():
    discharges = load_discharges()
    patients_data = load_patient_data()
    discharge_windows = build_discharge_windows(discharges)

    summarizer = Summarizer(model='gemini')

    results = []
    idx=0

    for _, case in discharges.iterrows():
        subject_id = case['subject_id']
        hadm_id = case['hadm_id']
        original_text = case['text']


        # Retrieve contextual data
        context_df = get_patient_window_data(
            subject_id=subject_id,
            hadm_id=hadm_id,
            windows_dict=discharge_windows,
            patient_df=patients_data,
            time_col='starttime'
        )
        context_json = context_builder_json(context_df)
        context_textual = context_builder_text(context_json)


        ### Summarization -------------------------------------------------------------
        summary_response = summarizer.summ(original_text)


        ### Decomposition -------------------------------------------------------------
        decomposition_prompt = DECOMPOSITION_PROMPT_TEMPLATE.format(
            summary_text=summary_response
        )
        decomp_response = summarizer.model.generate_content(decomposition_prompt).text


        ### Judging -------------------------------------------------------------------
        claims = [c.strip() for c in decomp_response.splitlines() if c.strip()]
        judged_claims_list = []
        judged_claims_text_list = []

        # Chunking with overlap
        premise = f"Discharge:\n{original_text}\nPatient context:\n{context_textual}"

        chunk_size = 2000          # text length per chunk
        overlap = 200              # number of overlapping characters
        premise_chunks = [
            premise[i : i + chunk_size]
            for i in range(0, len(premise), chunk_size - overlap)
        ]

        # Run for each 
        for claim in claims:

            best_label, best_score = None, 0.0
            for chunk in premise_chunks:
                result = nli(f"{chunk} </s> {claim}")
                label, score = result[0]["label"], result[0]["score"]

                if score > best_score:
                    best_label, best_score = label, score

                # Early stop
                if label == "ENTAILMENT" and score > 0.85:
                    break

            judged_claim = f"{claim}-{best_label}-{best_score:.3f}"
            judged_claims_list.append(judged_claim)

            judged_claim_text = f"{claim} - {best_label} - {best_score:.3f}"
            judged_claims_text_list.append(judged_claim_text)

        judged_claims_text = "\n".join(judged_claims_list)
        


        print('discharge: \n', original_text, '\nsummary:\n', summary_response,'\ndecomp:\n', decomp_response,'\ncontext:\n', context_textual, '\n*****\nJUDGED CLAIMS\n', judged_claims_text,'\n\n',)

        ## Claim Correction --------------------------------------------------------------------------------------


        ## Text Compiler ------------------------------------------------------------------------------------------
        prompt = '''
            You are a precise and concise medical writer.
            Your task is to merge and rewrite the following factual sentences into a single, well-structured text.

            Guidelines:
             - Only combine sentences if their meanings are fully equivalent or trivially overlapping.
             - Maintain all factual information — do not omit or change any details.
             - If there is any uncertainty about whether two sentences mean the same thing, keep both.
             - Preserve the order of the claims
             - Use a natural narrative tone, combining ideas logically (e.g., history -> reason for admission -> findings).
             - If multiple facts are related, integrate them into one flowing sentence.
             - Preserve medical accuracy, structure, and readability.
             - Do not speculate or interpret beyond what is written.


            Example 1:
            Input:
            The patient reported self-discontinuing her diuretics.
            The patient reported self-discontinuing Lasix.
            The patient reported self-discontinuing Spironolactone.
            The patient reported not adhering to a sodium-restricted diet.
            Self-discontinuing diuretics and not adhering to a sodium-restricted diet likely contributed to the worsening ascites.


            Output:
            The patient reported self-discontinuing her diuretics, including Lasix and Spironolactone, and not adhering to a sodium-restricted diet, both of which likely contributed to her worsening ascites.


            Example 2:
            Input:
            The patient has a history of HCV cirrhosis.
            The patient has a history of HIV on ART.
            The patient has a history of COPD.
            The patient has a history of bipolar disorder.
            The patient has a history of PTSD.

            Output:
            The patient has a history of HCV cirrhosis, HIV on ART, COPD, bipolar disorder, and PTSD.

            *NOW AGGREGATE (YOUR TASK)*:
            

        '''
        prompt = prompt + decomp_response

        enhanced_summary = summarizer.model.generate_content(prompt).text

        ## Compute Metrics ---------------------------------------------------------------------------------------
        # summary_response
        # enhanced_summary

        # BERT_score

        rouge = evaluate.load('rouge')
        rouge_scores = rouge.compute(
            predictions=[summary_response, enhanced_summary],
            references=[original_text, original_text],
            use_aggregator=False
        )


        results.append({
            "subject_id": subject_id,
            "hadm_id": hadm_id,
            "original_text": original_text,
            "context_textual": context_textual,
            "base_summary": summary_response,
            "decomposed_summary_text": judged_claims_text,
            "judged_claims": judged_claims_list,
            # "nli_results": nli_results
            "rouge_base_summary": {
                "rouge1": rouge_scores["rouge1"][0],
                "rouge2": rouge_scores["rouge2"][0],
                "rougeL": rouge_scores["rougeL"][0],
                "rougeLsum": rouge_scores["rougeLsum"][0],
            },
            "rouge_enhanced_summary": {
                "rouge1": rouge_scores["rouge1"][1],
                "rouge2": rouge_scores["rouge2"][1],
                "rougeL": rouge_scores["rougeL"][1],
                "rougeLsum": rouge_scores["rougeLsum"][1],
            },
        })


        results_df = pd.DataFrame(results)
        results_df.to_csv('results_df.csv', index=False)
        print(f"Checkpoint saved ({idx}/{len(discharges)}) → {'results_df.csv'}")
        idx += 1

    results_df = pd.DataFrame(results)
    results_df.to_csv('results_df.csv', index=False)
    return results_df


In [ ]:
decomp_judge_subpipeline()

In [ ]:
summarizer = Summarizer(model='gemini')

prompt = '''
    You are a careful medical summarizer.
    Your task is to merge and unify sentences that clearly describe the same information, while keeping all distinct clinical facts intact.

    Guidelines:
        - Only combine sentences if their meanings are fully equivalent or trivially overlapping.
        - Never remove or alter any factual information.
        - If there is any uncertainty about whether two sentences mean the same thing, keep both.
        - Preserve the order of the claims
        - Preserve medical accuracy, structure, and readability.
        - Output the result as a clear, concise list of sentences that retain all original meaning.


    Example 1:
    Input:
    The patient reported self-discontinuing her diuretics.
    The patient reported self-discontinuing Lasix.
    The patient reported self-discontinuing Spironolactone.
    The patient reported not adhering to a sodium-restricted diet.
    Self-discontinuing diuretics and not adhering to a sodium-restricted diet likely contributed to the worsening ascites.
    The patient has a history of HCV cirrhosis.
    The patient has a history of HIV on ART.
    The patient has a history of COPD.
    The patient has a history of bipolar disorder.
    The patient has a history of PTSD.

    Output:
    The patient has a history of HCV cirrhosis, HIV on ART, COPD, bipolar disorder, and PTSD. She reported self-discontinuing her diuretics, including Lasix and Spironolactone, and not adhering to a sodium-restricted diet, both of which likely contributed to her worsening ascites.


    Example 2:
    Input:
    The patient is female.
    The patient has a history of HIV on HAART.
    The patient has a history of COPD.
    The patient has a history of HCV cirrhosis.
    The patient's HCV cirrhosis is complicated by ascites.
    The patient's HCV cirrhosis is complicated by hepatic encephalopathy.
    The patient was admitted due to abdominal fullness.
    The patient was admitted due to abdominal discomfort.
    The patient had been recently discharged after a paracentesis.
    The patient experienced re-accumulation of ascites.
    The patient experienced increased abdominal distention.
    The patient experienced increased abdominal pain.
    The patient experienced visual hallucinations.
    The patient experienced forgetfulness.
    The patient's visual hallucinations are suggestive of hepatic encephalopathy.
    Upon discharge, the patient was independently ambulatory.
    Upon discharge, the patient had clear mental status.
    The patient's primary discharge diagnosis was diuretic refractory ascites.
    The patient's secondary discharge diagnosis was HCV cirrhosis.
    The patient's secondary discharge diagnosis was HIV.
    The patient's secondary discharge diagnosis was hyponatremia.
    The patient's secondary discharge diagnosis was COPD.
    The patient was instructed to continue her low-sodium diet.
    The patient was instructed to continue her fluid restriction.
    The patient was advised to return to the emergency room for any worsening abdominal pain.
    The patient was advised to return to the emergency room for any fever.
    The patient's discharge medications included her HIV medications.
    The patient's discharge medications included lactulose.
    The patient's discharge medications included rifaximin.
    The patient's discharge medications included Bactrim.

    Output:
    The patient is a female with a history of HCV cirrhosis complicated by ascites and hepatic encephalopathy, HIV on HAART, COPD, and hyponatremia. She was admitted with abdominal fullness and discomfort following a recent discharge after paracentesis, due to re-accumulation of ascites associated with increased abdominal distention and pain. She also experienced visual hallucinations and forgetfulness, findings suggestive of hepatic encephalopathy. Upon discharge, she was independently ambulatory and had a clear mental status. Her primary discharge diagnosis was diuretic-refractory ascites, with secondary diagnoses of HCV cirrhosis, HIV, hyponatremia, and COPD. She was instructed to continue a low-sodium diet and fluid restriction, and to return to the emergency room for any worsening abdominal pain or fever. Discharge medications included her HIV regimen, lactulose, rifaximin, and Bactrim.

    *NOW AGGREGATE (YOUR TASK)*:
    

'''

claims = '''

    The patient is female.
    The patient is ___ years old.
    The patient is healthy.
    The patient was admitted to Urology.
    The patient was admitted for a right renal mass.
    The right renal mass was suspicious for Renal Cell Carcinoma (RCC).
    The right renal mass was an incidental finding on MRI.
    The patient's past medical history includes nonspecific right axis deviation.
    The patient's past medical history includes a cesarean section.
    The patient has no known drug allergies.
    The patient's diet was gradually advanced.
    The remainder of the patient's hospital course was unremarkable.
    At discharge, the patient was in stable condition.
    At discharge, the patient was ambulating independently.
    At discharge, the patient was tolerating a diet.
    At discharge, the patient was voiding without difficulty.
    At discharge, the patient's pain was well-controlled on oral medication.
    Discharge medications included hydrocodone-acetaminophen for breakthrough pain.
    Discharge medications included docusate sodium for constipation.
    The patient was discharged home.
    The patient was instructed to avoid bathing.
    The patient was instructed to avoid swimming.
    The patient was instructed to limit lifting to under 10 pounds.
    The patient was instructed to avoid driving until her follow-up.
    The patient was instructed to use Tylenol as first-line pain relief.
    The patient was advised to resume home medications.
    The patient was advised to hold NSAIDs until follow-up.
    The patient was advised to seek medical attention for fevers above 101.5 F.
    The patient was advised to seek medical attention for vomiting.
    The patient was advised to seek medical attention for worsening incision.
    A follow-up appointment with Urology in 3 weeks was scheduled.
    The patient's final diagnosis was renal cell carcinoma.

'''

prompt = prompt + claims

output = summarizer.model.generate_content(prompt).text


print(output)

In [ ]:
summarizer = Summarizer(model='gemini')



In [ ]:
pd.DataFrame(data)

In [ ]:
complete_pipeline()

In [ ]:
claim_judge_pipeline()

In [ ]:
data_preparation()

In [ ]:
generate_summary_database()

In [ ]:
d = load_clinical_context()
d

In [ ]:
print(d.keys())
d['billed_procedures']

In [ ]:
d['billed_procedures']['long_title'].value_counts()